In [9]:
pip install langchain -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [7]:
pip install  openai langchain sentence_transformers chromadb unstructured -q


Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install -U langchain-community langchain-huggingface -q


Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install "unstructured[docx]" -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import DirectoryLoader

directory = r'/Users/jkim/Library/CloudStorage/OneDrive-OmniAb,Inc/Documents/xPloration User Manual/xPloration User Manual'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)


/Users/jkim/project_backend/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

60


In [3]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/Users/jkim/project_backend/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/jkim/project_backend/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

In [6]:
query = "Guided Scan mode"
matching_docs = db.similarity_search(query)

matching_docs[0]

Document(metadata={'source': '/Users/jkim/Library/CloudStorage/OneDrive-OmniAb,Inc/Documents/xPloration User Manual/xPloration User Manual/Recent_xPloration User Manual_2024-03-25.docx'}, page_content='Guided Scan\n\nAbout Guided Scan\n\nGuided Scan marks the start of the complete workflow. It allows you to input scanning details, insert a chip, adjust imaging parameters, select a region, and execute automated scanning.\n\nProcedure\n\nOn the main screen, click GUIDED SCAN to go to the next screen.')

In [7]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

vectordb.persist()


/Users/jkim/project_backend/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [16]:
documents

[Document(metadata={'source': '/Users/jkim/Library/CloudStorage/OneDrive-OmniAb,Inc/Documents/xPloration User Manual/xPloration User Manual/Recent_xPloration User Manual_2024-03-25.docx'}, page_content="xPloration® User Guide\n\nUser Manual\n\nUM-xPloration-00001 Rev D\t2\tOmniAb, Inc.\n\nUM-xPloration-00001 Rev A\t23\tConfidential\n\nxPloration® User Guide\n\nTracking Information\n\nItem Run 1 Run 2 Run 3 Run 4 Run 5 Run 6 Project ID Screening Date \n(YYYY-MM-DD) Operator Chip Number Animal Number Antigen Target Fluorescent Channels (Exposure Time) Screening Bead/Cell Types Number of Capillaries Screened Number of Hits Extraction Plate ID and # of Extractions\n\nUM-xPloration-00001 Rev A\t23\tConfidential\n\nRevision History\n\nRev Date Author Change A 2/21/2024 John Kim Initial release\n\nContents\n\nIntroduction\n\nAbout This Manual\n\nThis manual describes how to run the xPloration® workflow using the xPloration system. It provides an overview of how to prepare samples, operate the

OpenAI

In [9]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

In [25]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = "What is excitationn and emisssion of GPF filter?"
matching_docs = db.similarity_search(query)
answer = chain.run(input_documents=matching_docs, question=query)
answer




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
xPloration Filter Settings

Cube Excitation (nm) Emission (nm) Transmission Graph UV 360–390 430–490 GFP 450–490 500–550 TRITC* 535–557 570–639 Cy5 (Red) 590–645 665–733

xPloration v2 only

for focusing. The default step is set to 10 μm but can be adjusted for finer or coarser steps. Each point must be adjusted at least once for a green checkmark to appear. Ensure that all five points display green checkmarks. Start the scanning process by clicking RUN. IMPORTANT: Ensure precise focusing on each point using the primary channel. Autoscan Using the predetermined image settings, the instrument proceeds to image the specified sample region. You are provided with an estimated duration and percentage of s

'The excitation for GFP filter is 450-490 nm, and the emission is 500-550 nm.'

In [26]:
from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'The excitation range for GFP filter is 450-490 nm, and the emission range is 500-550 nm.'